In [1]:
!pip install transformers
!pip install torch

     |████████████████████████████████| 4.0 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 880 kB 43.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 52.5 MB/s eta 0:00:01
     |████████████████████████████████| 759 kB 16.4 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895252 sha256=1d40cf3bf23caa0977c39e3595f25acc0dc555f75b18a480c78721156343764a
  Stored in directory: /root/.cache/pip/wheels/4c/64/31/e9900a234b23fb3e9dc565d6114a9d6ff84a72dbdd356502b4
Successfully built sacremoses
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

huggingface-hub 0.4.0 requires packaging>=20.9, but you'll have p

In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
from tqdm import tqdm

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [5]:
titles=list(df.columns)
titles[1], titles[2]=titles[2], titles[1]
df=df[titles]
df.head()

,id,comment_text,target,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,Is this something I'll be able to install on m...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,haha you guys are a bunch of losers.,0.893617,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [6]:
train_df, val_df = train_test_split(df, test_size=0.05)
train_df.shape, val_df.shape

((1714630, 45), (90244, 45))

In [7]:
train_toxic = train_df[train_df['target']>=0.5]
train_clean = train_df[train_df['target']< 0.5]
print(len(train_toxic))
print(len(train_clean))
val_toxic = val_df[val_df['target']>=0.5]
val_clean = val_df[val_df['target']< 0.5]
print(len(val_toxic))
print(len(val_clean))

137128
1577502
7206
83038


In [8]:
train_df = pd.concat([
  train_toxic.sample(10000),
  train_clean.sample(10000)
])
val_df = pd.concat([
  val_toxic.sample(1000),
  val_clean.sample(1000)
])
train_df.shape, val_df.shape

((20000, 45), (2000, 45))

In [9]:
target_cols=train_df.columns.tolist()[2:8]
target_cols

['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

In [10]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
train_encodings = tokenizer(list(train_df["comment_text"]), truncation=True, padding=True)
val_encodings = tokenizer(list(val_df["comment_text"]), truncation=True, padding=True)

In [12]:
import torch

class JigsawDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = JigsawDataset(train_encodings, train_df.iloc[:,2:8].values)
val_dataset = JigsawDataset(val_encodings, val_df.iloc[:,2:8].values)

In [13]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=True)

In [14]:
from transformers import BertForSequenceClassification
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
import torch.nn as nn

In [15]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6, output_attentions=False, output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
optimizer = AdamW(model.parameters(), lr=4e-5)
loss_fn = BCEWithLogitsLoss()

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [18]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
model.to(device)

for epoch in range(50):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for batch in val_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(**inputs, labels=labels)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(torch.argmax(labels, dim=1).cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=1)
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f"Validation accuracy - epoch_{epoch}: {accuracy:.4f}")
    print(f"Precision score - epoch_{epoch}: {precision:.4f}")
    print(f"Recall score - epoch_{epoch}: {recall:.4f}")
    print(f"F1 score - epoch_{epoch}: {f1:.4f}")

Validation accuracy - epoch_0: 0.9205
Precision score - epoch_0: 0.4115
Recall score - epoch_0: 0.9205
F1 score - epoch_0: 0.9163
Validation accuracy - epoch_1: 0.9170
Precision score - epoch_1: 0.4391
Recall score - epoch_1: 0.9170
F1 score - epoch_1: 0.9186
Validation accuracy - epoch_2: 0.6710
Precision score - epoch_2: 0.4791
Recall score - epoch_2: 0.6710
F1 score - epoch_2: 0.7677
Validation accuracy - epoch_3: 0.8955
Precision score - epoch_3: 0.2464
Recall score - epoch_3: 0.8955
F1 score - epoch_3: 0.9056
Validation accuracy - epoch_4: 0.8940
Precision score - epoch_4: 0.4171
Recall score - epoch_4: 0.8940
F1 score - epoch_4: 0.9045
Validation accuracy - epoch_5: 0.8130
Precision score - epoch_5: 0.2387
Recall score - epoch_5: 0.8130
F1 score - epoch_5: 0.8598
Validation accuracy - epoch_6: 0.8645
Precision score - epoch_6: 0.4360
Recall score - epoch_6: 0.8645
F1 score - epoch_6: 0.8905
Validation accuracy - epoch_7: 0.8945
Precision score - epoch_7: 0.2382
Recall score - epo

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation accuracy - epoch_10: 0.8525
Precision score - epoch_10: 0.1716
Recall score - epoch_10: 0.8525
F1 score - epoch_10: 0.8803
Validation accuracy - epoch_11: 0.8240
Precision score - epoch_11: 0.2114
Recall score - epoch_11: 0.8240
F1 score - epoch_11: 0.8653


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation accuracy - epoch_12: 0.7655
Precision score - epoch_12: 0.1898
Recall score - epoch_12: 0.7655
F1 score - epoch_12: 0.8303
Validation accuracy - epoch_13: 0.6425
Precision score - epoch_13: 0.2185
Recall score - epoch_13: 0.6425
F1 score - epoch_13: 0.7467
Validation accuracy - epoch_14: 0.8035
Precision score - epoch_14: 0.2158
Recall score - epoch_14: 0.8035
F1 score - epoch_14: 0.8528


...

Validation accuracy - epoch_46: 0.6715
Precision score - epoch_46: 0.2019
Recall score - epoch_46: 0.6715
F1 score - epoch_46: 0.7678


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation accuracy - epoch_47: 0.7485
Precision score - epoch_47: 0.1891
Recall score - epoch_47: 0.7485
F1 score - epoch_47: 0.8197


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation accuracy - epoch_48: 0.7575
Precision score - epoch_48: 0.1797
Recall score - epoch_48: 0.7575
F1 score - epoch_48: 0.8254
Validation accuracy - epoch_49: 0.7655
Precision score - epoch_49: 0.2415
Recall score - epoch_49: 0.7655
F1 score - epoch_49: 0.8318


In [19]:
from scipy.stats import ttest_rel

accuracy_run1 = [0.85, 0.88, 0.82]
precision_run1 = [0.84, 0.87, 0.81]
recall_run1 = [0.86, 0.89, 0.83]
f1_run1 = [0.82, 0.85, 0.80]

accuracy_run2 = [0.88, 0.90, 0.87]
precision_run2 = [0.87, 0.89, 0.86]
recall_run2 = [0.89, 0.91, 0.88]
f1_run2 = [0.85, 0.88, 0.84]

t_stat_accuracy, p_value_accuracy = ttest_rel(accuracy_run1, accuracy_run2)

t_stat_precision, p_value_precision = ttest_rel(precision_run1, precision_run2)

t_stat_recall, p_value_recall = ttest_rel(recall_run1, recall_run2)

t_stat_f1, p_value_f1 = ttest_rel(f1_run1, f1_run2)

print(f"Accuracy t-statistic: {t_stat_accuracy}, p-value: {p_value_accuracy}")
print(f"Precision t-statistic: {t_stat_precision}, p-value: {p_value_precision}")
print(f"Recall t-statistic: {t_stat_recall}, p-value: {p_value_recall}")
print(f"F1 t-statistic: {t_stat_f1}, p-value: {p_value_f1}")


Accuracy t-statistic: -3.7796447300922718, p-value: 0.06341418841830604
Precision t-statistic: -3.7796447300922824, p-value: 0.06341418841830572
Recall t-statistic: -3.7796447300922718, p-value: 0.06341418841830604
F1 t-statistic: -10.0000000000001, p-value: 0.009852457023325498


In [20]:
text = "I will kill you"
encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
logits = outputs.logits

probabilities = torch.sigmoid(logits)

print(probabilities)

tensor([[0.8630, 0.1100, 0.0060, 0.0043, 0.0656, 0.8722]], device='cuda:0')


In [21]:
probabilities = probabilities.cpu()
array = probabilities.detach().numpy()
array

array([[0.86297333, 0.10996056, 0.00602231, 0.00429804, 0.06555375,
        0.87218744]], dtype=float32)

In [22]:
array=array.flatten()

In [23]:
for label, prediction in zip(target_cols, array):
  print(f"{label}: {prediction}")

target: 0.8629733324050903
severe_toxicity: 0.10996056348085403
obscene: 0.006022311747074127
identity_attack: 0.0042980448342859745
insult: 0.06555374711751938
threat: 0.8721874356269836
